In [1]:
import pandas as pd
df_train = pd.read_csv(r"P:\Ahmed\Datasets\14825193\dataset.csv")


In [2]:
df_train 

,filestem,patient_id,study_number,timehash,gender,age,laterality,projection,initial_exam,ao_classification,cast,diagnosis_uncertain,osteopenia,fracture_visible,metal,pixel_spacing,device_manufacturer
0,0001_1297860395_01_WRI-L1_M014,1,1,1297860395,M,14.1,L,1,1.0,23r-M/2.1,NaN,NaN,NaN,NaN,NaN,0.144,Siemens
1,0001_1297860435_01_WRI-L2_M014,1,1,1297860435,M,14.1,L,2,1.0,23r-M/2.1,NaN,NaN,NaN,1.0,NaN,0.144,Siemens
2,0002_0354485735_01_WRI-R1_F012,2,1,354485735,F,12.0,R,1,1.0,23r-M/2.1,NaN,1.0,NaN,NaN,NaN,0.144,Siemens
3,0002_0354485759_01_WRI-R2_F012,2,1,354485759,F,12.0,R,2,1.0,23r-M/2.1,NaN,1.0,NaN,NaN,NaN,0.144,Siemens
4,0003_0662359226_01_WRI-R1_M011,3,1,662359226,M,11.1,R,1,1.0,23-M/3.1,NaN,NaN,NaN,1.0,NaN,0.144,Siemens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20322,6091_0590289015_02_WRI-L2_F008,6091,2,590289015,F,8.7,L,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.144,Siemens
20323,6092_0295935784_01_WRI-L1_M003,6092,1,295935784,M,3.5,L,1,1.0,22-D/1.1,NaN,NaN,NaN,NaN,NaN,0.144,Siemens
20324,6092_0295935837_01_WRI-L2_M003,6092,1,295935837,M,3.5,L,2,1.0,22-D/1.1,NaN,NaN,NaN,NaN,NaN,0.144,Siemens
20325,6093_0885712402_01_WRI-L1_M008,6093,1,885712402,M,8.4,L,1,1.0,23r-M/3.1,NaN,NaN,NaN,1.0,NaN,0.144,Siemens


In [6]:
import os
from textwrap import shorten
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import tqdm
import shutil



img_dir = r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\images"
ann_dir = r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\labels"
df = pd.read_csv(r"P:\Ahmed\Datasets\14825193\dataset.csv")

splitter1 = GroupShuffleSplit(test_size=.3, n_splits=2)
split = splitter1.split(df, groups=df["patient_id"])
train_idxs, valid_idxs = next(split)
train_df = df.iloc[train_idxs]
temp_df = df.iloc[valid_idxs]

splitter2 = GroupShuffleSplit(test_size=.33333, n_splits=2)
split = splitter2.split(temp_df, groups=temp_df["patient_id"])
valid_idxs, test_idxs = next(split)
valid_df = temp_df.iloc[valid_idxs]
test_df = temp_df.iloc[test_idxs]

train_df.to_csv(r"P:\Ahmed\Datasets\14825193\CSVs\train_data.csv", index=False)
valid_df.to_csv(r"P:\Ahmed\Datasets\14825193\CSVs\valid_data.csv", index=False)
test_df.to_csv(r"P:\Ahmed\Datasets\14825193\CSVs\test_data.csv", index=False)

img_train_dir = r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\images\train"
img_valid_dir = r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\images\valid"
img_test_dir = r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\images\test"
ann_train_dir = r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\labels\train"
ann_valid_dir = r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\labels\valid"
ann_test_dir = r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\labels\test"
for dir in [img_train_dir, img_valid_dir, img_test_dir, ann_train_dir, ann_valid_dir, ann_test_dir]:
    if os.path.exists(dir) == False:
        os.makedirs(dir)

for i in tqdm.tqdm(train_df.index, total=len(train_df)):
    filestem = train_df.loc[i, "filestem"]
    shutil.move(os.path.join(img_dir, filestem + ".png"), os.path.join(img_train_dir, filestem + ".png"))
    shutil.move(os.path.join(ann_dir, filestem + ".txt"), os.path.join(ann_train_dir, filestem + ".txt"))

for i in tqdm.tqdm(valid_df.index, total=len(valid_df)):
    filestem = valid_df.loc[i, "filestem"]
    shutil.move(os.path.join(img_dir, filestem + ".png"), os.path.join(img_valid_dir, filestem + ".png"))
    shutil.move(os.path.join(ann_dir, filestem + ".txt"), os.path.join(ann_valid_dir, filestem + ".txt"))

for i in tqdm.tqdm(test_df.index, total=len(test_df)):
    filestem = test_df.loc[i, "filestem"]
    shutil.move(os.path.join(img_dir, filestem + ".png"), os.path.join(img_test_dir, filestem + ".png"))
    shutil.move(os.path.join(ann_dir, filestem + ".txt"), os.path.join(ann_test_dir, filestem + ".txt"))

N = len(df)
print("Data split compleated according to PatientID:")
print(f"  - {len(train_df)} ({100 * len(train_df)/N:.3f}%) images in the training set.")
print(f"  - {len(valid_df)} ({100 * len(valid_df)/N:.3f}%) images in the validation set.")
print(f"  - {len(test_df)} ({100 * len(test_df)/N:.3f}%) images in the testing set.")

100%|██████████| 2035/2035 [00:01<00:00, 1724.31it/s]

Data split compleated according to PatientID:
  - 14126 (69.494%) images in the training set.
  - 4166 (20.495%) images in the validation set.
  - 2035 (10.011%) images in the testing set.


In [2]:
from ultralytics import YOLO
import torch 
torch.cuda.set_device(0)

In [2]:
model = YOLO('yolov8x.pt')

In [3]:

model.train(data=r"P:\Ahmed\Datasets\14825193\folder_structure\yolov5\meta.yaml",epochs=50,imgsz = 640,device=0,batch=-1)

Ultralytics YOLOv8.2.48  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=P:\Ahmed\Datasets\14825193\folder_structure\yolov5\meta.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

train: Scanning P:\Ahmed\Datasets\14825193\folder_structure\yolov5\labels\train.cache... 14126 images, 11 backgrounds, 0 corrupt: 100%|██████████| 14126/14126 [00:00<?, ?it/s]
val: Scanning P:\Ahmed\Datasets\14825193\folder_structure\yolov5\labels\valid.cache... 4166 images, 4 backgrounds, 0 corrupt: 100%|██████████| 4166/4166 [00:00<?, ?it/s]


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      3.46G      1.367      1.416      1.256          8        640: 100%|██████████| 7063/7063 [23:19<00:00,  5.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1042/1042 [01:46<00:00,  9.75it/s]

                   all       4166       9914      0.721      0.335      0.374      0.231


...

                   all       4166       9914      0.662      0.528      0.534      0.343



50 epochs completed in 30.314 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 136.7MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 136.7MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 268 layers, 68132235 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1042/1042 [02:46<00:00,  6.26it/s]


                   all       4166       9914      0.522       0.58      0.569       0.36
           boneanomaly         69         97      0.192      0.186      0.113      0.044
            bonelesion          8          8       0.64       0.23      0.403      0.201
           foreignbody          2          2          0          0          0          0
              fracture       2787       3709       0.82       0.92      0.941      0.563
                 metal        201        225       0.83      0.898      0.914      0.775
    periostealreaction        494        786      0.459      0.784      0.698      0.353
          pronatorsign        113        113      0.492      0.823      0.739      0.394
            softtissue        110        114      0.313      0.395      0.326      0.172
                  text       4151       4860       0.95      0.987       0.99      0.743
Speed: 0.3ms preprocess, 32.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\detec

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020F2AD7A7D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.

In [3]:
from ultralytics import YOLO
import torch 
torch.cuda.set_device(0)
model = YOLO(r"P:\Ahmed\Models\wrist\runs\detect\train2\weights\best.pt")
result = model(r"P:\Api\archive (3)\Bone Break Classification\Bone Break Classification\Greenstick fracture\Train\greenstick-fracture-involving-radius_big_gallery_jpeg.rf.5e627ee89a3fcb6e09091f3a1cc6ab26.jpg",show=True,save=True)


image 1/1 P:\Api\archive (3)\Bone Break Classification\Bone Break Classification\Greenstick fracture\Train\greenstick-fracture-involving-radius_big_gallery_jpeg.rf.5e627ee89a3fcb6e09091f3a1cc6ab26.jpg: 640x640 1 fracture, 51.8ms
Speed: 3.0ms preprocess, 51.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict13


: 